In [105]:
import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)

In [3]:
import pandas as pd
from cpg_utils import to_path
from pandas_plink import read_plink1_bin

In [96]:
phenotype_file = "gs://cpg-tob-wgs-test/scrna-seq/grch38_association_files/expression_files/B_naive_expression.tsv"

In [106]:
phenotype = pd.read_csv(phenotype_file, sep="\t", index_col=0)

In [98]:
phenotype = xr.DataArray(
    phenotype.values,
    dims=['sample', 'gene'],
    coords={'sample': phenotype.index.values, 'gene': phenotype.columns.values},
)

In [99]:
phenotype

<xarray.DataArray (sample: 969, gene: 27817)>
array([[ 0.        ,  0.        ,  0.        , ...,  1.9380531 ,
         0.11504425,  8.38053097],
       [ 0.        ,  0.        ,  0.        , ...,  1.79591837,
         0.10204082,  7.40816327],
       [ 0.        ,  0.        ,  0.02040816, ...,  1.83673469,
         0.10204082,  6.08163265],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  2.25      ,
         0.15      , 10.5       ],
       [ 0.        ,  0.        ,  0.        , ...,  2.22857143,
         0.02857143, 10.65714286],
       [ 0.        ,  0.        ,  0.        , ...,  2.        ,
         0.15789474,  9.68421053]])
Coordinates:
  * sample   (sample) object '686_687' '682_683' ... '814_815' '821_822'
  * gene     (gene) object 'RP11-34P13.7' 'RP11-34P13.8' ... 'MT-ND6' 'MT-CYB'

In [101]:
genotype_file_bed = "gs://cpg-tob-wgs-test/v0/plink_files/vpreb3_rare_promoter.bed"
genotype_file_bim = "gs://cpg-tob-wgs-test/v0/plink_files/vpreb3_rare_promoter.bim"
genotype_file_fam = "gs://cpg-tob-wgs-test/v0/plink_files/vpreb3_rare_promoter.fam"

In [107]:
# bed
with to_path(genotype_file_bed).open('rb') as handle:
    data = handle.readlines()
with open('temp.bed', 'wb') as handle:
    handle.writelines(data)
# bim
with to_path(genotype_file_bim).open('rb') as handle:
    data = handle.readlines()
with open('temp.bim', 'wb') as handle:
    handle.writelines(data)
# fam
with to_path(genotype_file_fam).open('rb') as handle:
    data = handle.readlines()
with open('temp.fam', 'wb') as handle:
    handle.writelines(data)
# read
G = read_plink1_bin('temp.bed')

In [14]:
sample_mapping_file = "gs://cpg-tob-wgs-test/scrna-seq/grch38_association_files/OneK1K_CPG_IDs.tsv"

In [108]:
sample_mapping = pd.read_csv(sample_mapping_file, sep="\t")

In [29]:
sample_mapping.head()

,OneK1K_ID,InternalID,ExternalID
0,926_927,CPG9787,TOB1801
1,930_931,CPG9829,TOB1805
2,946_947,CPG9985,TOB1821
3,943_944,CPG9951,TOB1818
4,947_948,CPG9993,TOB1822


In [178]:
sample_mapping[sample_mapping['OneK1K_ID'] == '943_944']

,OneK1K_ID,InternalID,ExternalID
3,943_944,CPG9951,TOB1818


In [109]:
## samples with expression data 
donors_onek1k = sample_mapping['OneK1K_ID'].unique()
donors_onek1k.sort()
donors_exprs = sorted(set(list(phenotype.sample.values)).intersection(donors_onek1k))
logging.info('Number of unique donors with expression data: {}'.format(len(donors_exprs)))

2022-09-28 00:42:04 INFO (root 5): Number of unique donors with expression data: 969


In [110]:
## samples with genotype data 
donors_cpg = sample_mapping['InternalID'].unique()
donors_cpg.sort()
donors_geno = sorted(set(list(G.sample.values)).intersection(donors_cpg))
logging.info('Number of unique donors with genotype data: {}'.format(len(donors_geno)))

2022-09-28 00:42:23 INFO (root 5): Number of unique donors with genotype data: 899


In [126]:
sample_mapping1 = sample_mapping.loc[sample_mapping['OneK1K_ID'].isin(donors_exprs)]
sample_mapping1.shape

(969, 3)

In [127]:
sample_mapping2 = sample_mapping1.loc[sample_mapping1['InternalID'].isin(donors_geno)]
sample_mapping2.shape

(887, 3)

In [129]:
sample_mapping_both = sample_mapping2

In [133]:
sample_mapping_both

,OneK1K_ID,InternalID,ExternalID
0,926_927,CPG9787,TOB1801
1,930_931,CPG9829,TOB1805
2,946_947,CPG9985,TOB1821
3,943_944,CPG9951,TOB1818
4,947_948,CPG9993,TOB1822
...,...,...,...
977,929_930,CPG9811,TOB1804
978,935_936,CPG9878,TOB1810
979,937_938,CPG9894,TOB1812
981,961_962,CPG10132,TOB1836


NameError: name 'sample_key_df' is not defined

In [135]:
donors_e = sample_mapping_both['OneK1K_ID'].unique()
donors_g = sample_mapping_both['InternalID'].unique()
assert(len(donors_e)==len(donors_g))
# len(donors_e)

In [137]:
kinship_file = 'gs://cpg-tob-wgs-test/v0/skat/grm_wide.csv'

In [139]:
## read in GRM (genotype relationship matrix; kinship matrix)
K = pd.read_csv(kinship_file, index_col=0)
K.index = K.index.astype('str')
assert all(K.columns == K.index)  # symmetric matrix, donors x donors

In [140]:
K = xr.DataArray(
    K.values,
    dims=['sample_0', 'sample_1'],
    coords={'sample_0': K.columns, 'sample_1': K.index},
)
K = K.sortby('sample_0').sortby('sample_1')

In [141]:
K.shape

(1086, 1086)

In [144]:
K.sample_0.values

array(['1', '10', '100', ..., '997', '998', '999'], dtype=object)

In [166]:
import re
donors_e_short = [re.sub(".*_", "", donor) for donor in donors_e]
donors_e_short[1:5]

['931', '947', '944', '948']

In [167]:
donors_e[1:5]

array(['930_931', '946_947', '943_944', '947_948'], dtype=object)

In [168]:
donors_k = sorted(set(list(K.sample_0.values)).intersection(donors_e_short))

In [169]:
len(donors_k)

887

In [170]:
K = K.sel(sample_0=donors_k, sample_1=donors_k)
assert all(K.sample_0 == donors_k)

In [171]:
assert all(K.sample_1 == donors_k)

In [172]:
K.shape

(887, 887)

In [173]:
data = K.values
K_df = pd.DataFrame(data, columns=K.sample_0, index=K.sample_1)
K_df.head()

,1,10,1000,1001,1003,1004,1005,1006,1007,1008,...,98,980,985,986,987,99,991,992,998,999
1,0.995624,0.001088,-0.002613,0.001665,-0.005944,-0.000997,-0.005255,0.000822,-0.008134,0.005964,...,-0.001763,-0.004104,-0.003397,-0.002764,0.000806,-0.004308,0.004359,-0.001213,-0.002835,0.000625
10,0.001088,0.990255,-0.001234,0.001762,0.001406,-0.002503,-0.007511,0.003694,-0.003851,-0.000265,...,-0.003144,-0.002243,0.002729,-0.006724,0.005370,0.012675,0.001112,-0.004417,-0.002750,-0.002998
1000,-0.002613,-0.001234,1.002540,-0.001320,-0.003663,-0.001874,0.006776,-0.003634,0.000390,-0.004447,...,0.001943,-0.002532,0.001785,-0.002523,0.001729,-0.002736,-0.001694,-0.004914,-0.005415,-0.007256
1001,0.001665,0.001762,-0.001320,1.013439,0.002794,0.002284,-0.002669,-0.001133,-0.003800,0.000527,...,0.001164,0.003286,-0.005132,-0.004572,0.000253,0.000812,-0.004417,-0.004742,-0.003498,0.005071
1003,-0.005944,0.001406,-0.003663,0.002794,0.999531,-0.003343,-0.000980,0.003208,0.004163,0.007510,...,-0.002012,-0.000914,-0.004123,0.003967,-0.002244,-0.003944,-0.000962,-0.006314,-0.006447,-0.000345


In [57]:
donors0 = sample_mapping['OneK1K_ID'].unique()
donors0.sort()
len(donors0)

983

In [31]:
# donors0 = sample_mapping['ExternalID'].unique()
# donors0.sort()
# len(donors0)

983

In [32]:
# donors0 = sample_mapping['InternalID'].unique()
# donors0.sort()
# len(donors0)

983

In [58]:
y_file = "gs://cpg-tob-wgs-test/scrna-seq/grch38_association_files/expression_files/B_naive_expression.tsv"
# y_file = "gs://cpg-tob-wgs-test/scrna-seq/grch38_association_files/expression_files/B_memory_expression.tsv"
# y_file = "gs://cpg-tob-wgs-test/scrna-seq/grch38_association_files/expression_files/B_intermediate_expression.tsv"

In [100]:
# phenotype = pd.read_csv(y_file, sep="\t", index_col=0)
# phenotype.head()

In [60]:
import xarray as xr

In [61]:
phenotype = xr.DataArray(
        phenotype.values,
        dims=['sample', 'gene'],
        coords={'sample': phenotype.index.values, 'gene': phenotype.columns.values},
    )

In [62]:
phenotype

<xarray.DataArray (sample: 969, gene: 27817)>
array([[ 0.        ,  0.        ,  0.        , ...,  1.9380531 ,
         0.11504425,  8.38053097],
       [ 0.        ,  0.        ,  0.        , ...,  1.79591837,
         0.10204082,  7.40816327],
       [ 0.        ,  0.        ,  0.02040816, ...,  1.83673469,
         0.10204082,  6.08163265],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  2.25      ,
         0.15      , 10.5       ],
       [ 0.        ,  0.        ,  0.        , ...,  2.22857143,
         0.02857143, 10.65714286],
       [ 0.        ,  0.        ,  0.        , ...,  2.        ,
         0.15789474,  9.68421053]])
Coordinates:
  * sample   (sample) object '686_687' '682_683' ... '814_815' '821_822'
  * gene     (gene) object 'RP11-34P13.7' 'RP11-34P13.8' ... 'MT-ND6' 'MT-CYB'

In [65]:
phenotype = phenotype.sel(sample=donors)
phenotype

<xarray.DataArray (sample: 969, gene: 27817)>
array([[ 0.        ,  0.        ,  0.        , ...,  2.39423077,
         0.07692308,  8.60576923],
       [ 0.        ,  0.        ,  0.        , ...,  1.73529412,
         0.07352941,  7.63235294],
       [ 0.        ,  0.        ,  0.        , ...,  2.71276596,
         0.08510638,  9.25531915],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  2.33707865,
         0.08988764, 10.43820225],
       [ 0.        ,  0.        ,  0.        , ...,  1.92857143,
         0.05102041,  6.66326531],
       [ 0.        ,  0.        ,  0.        , ...,  2.38461538,
         0.1025641 ,  7.41025641]])
Coordinates:
  * sample   (sample) object '1000_1001' '1001_1002' ... '99_99' '9_9'
  * gene     (gene) object 'RP11-34P13.7' 'RP11-34P13.8' ... 'MT-ND6' 'MT-CYB'

In [66]:
gene_name = "VPREB3"

In [67]:
y = phenotype.sel(gene=gene_name)

In [68]:
y

<xarray.DataArray (sample: 969)>
array([0.97115385, 0.97058824, 1.14893617, 1.25806452, 1.09375   ,
       1.46153846, 1.05      , 1.17391304, 0.86792453, 0.69444444,
       0.84745763, 1.14634146, 0.76237624, 1.23493976, 1.25974026,
       1.38095238, 0.89312977, 0.4       , 0.51648352, 1.18085106,
       0.7       , 1.13157895, 1.00970874, 1.03636364, 0.93589744,
       1.20689655, 0.83870968, 1.04878049, 0.70588235, 0.75862069,
       1.81818182, 1.13333333, 1.23333333, 1.25925926, 1.2       ,
       0.44      , 0.95454545, 1.1509434 , 1.33333333, 1.15      ,
       1.08      , 0.96428571, 0.93043478, 1.3255814 , 1.31578947,
       1.11827957, 1.03947368, 1.27272727, 1.26086957, 1.58139535,
       0.62      , 1.22727273, 1.18309859, 1.53333333, 1.03773585,
       1.27272727, 0.75      , 1.07246377, 1.03571429, 0.93103448,
       1.59375   , 1.05970149, 1.28723404, 1.22360248, 1.14492754,
       1.33333333, 1.33333333, 1.20454545, 1.12820513, 1.66666667,
       0.34545455, 0.49230769, 0.78461538, 1.06666667, 1.2       ,
       1.        , 1.5       , 0.89361702, 1.22857143, 0.69444444,
       0.75384615, 0.7       , 0.6875    , 0.96969697, 0.64285714,
       1.        , 0.52459016, 0.76785714, 0.71052632, 1.14782609,
       0.36842105, 0.71171171, 0.68      , 0.42857143, 0.54166667,
       0.55629139, 0.4       , 0.36666667, 0.70945946, 0.5       ,
...
       0.95652174, 0.66666667, 0.71428571, 1.16666667, 1.42372881,
       0.85365854, 1.1025641 , 0.90909091, 2.625     , 0.94160584,
       1.22222222, 1.        , 0.94871795, 0.43137255, 1.60135135,
       1.08      , 1.        , 1.17808219, 0.75641026, 0.67857143,
       0.8125    , 1.13253012, 0.90697674, 0.88461538, 1.20833333,
       0.97938144, 1.10576923, 1.3559322 , 1.0619469 , 0.5       ,
       1.2375    , 0.98809524, 0.83157895, 0.8       , 0.98333333,
       0.73015873, 0.5890411 , 0.79411765, 0.54347826, 0.4       ,
       0.60747664, 0.38095238, 0.21978022, 0.56875   , 0.59459459,
       0.3164557 , 0.79166667, 1.05357143, 0.        , 1.05228758,
       0.5       , 1.23417722, 0.71830986, 0.56481481, 0.97716895,
       0.79245283, 0.82758621, 0.65116279, 0.83333333, 0.86666667,
       0.71779141, 1.15384615, 1.01388889, 0.72580645, 0.86842105,
       1.        , 0.7       , 1.84210526, 1.22222222, 1.37931034,
       0.91666667, 0.98876404, 0.69747899, 1.0052356 , 0.81578947,
       1.125     , 0.64705882, 0.76056338, 0.74576271, 0.87777778,
       0.92142857, 1.03333333, 0.65384615, 0.92307692, 1.        ,
       1.1625    , 0.62637363, 1.10714286, 1.07352941, 1.09375   ,
       1.14285714, 0.86440678, 1.30645161, 0.8071066 , 0.87878788,
       1.        , 0.80898876, 0.89795918, 0.58974359])
Coordinates:
  * sample   (sample) object '1000_1001' '1001_1002' ... '99_99' '9_9'
    gene     <U6 'VPREB3'

In [70]:
y_c = y.values.reshape(y.shape[0], 1)

In [73]:
y

<xarray.DataArray (sample: 969)>
array([0.97115385, 0.97058824, 1.14893617, 1.25806452, 1.09375   ,
       1.46153846, 1.05      , 1.17391304, 0.86792453, 0.69444444,
       0.84745763, 1.14634146, 0.76237624, 1.23493976, 1.25974026,
       1.38095238, 0.89312977, 0.4       , 0.51648352, 1.18085106,
       0.7       , 1.13157895, 1.00970874, 1.03636364, 0.93589744,
       1.20689655, 0.83870968, 1.04878049, 0.70588235, 0.75862069,
       1.81818182, 1.13333333, 1.23333333, 1.25925926, 1.2       ,
       0.44      , 0.95454545, 1.1509434 , 1.33333333, 1.15      ,
       1.08      , 0.96428571, 0.93043478, 1.3255814 , 1.31578947,
       1.11827957, 1.03947368, 1.27272727, 1.26086957, 1.58139535,
       0.62      , 1.22727273, 1.18309859, 1.53333333, 1.03773585,
       1.27272727, 0.75      , 1.07246377, 1.03571429, 0.93103448,
       1.59375   , 1.05970149, 1.28723404, 1.22360248, 1.14492754,
       1.33333333, 1.33333333, 1.20454545, 1.12820513, 1.66666667,
       0.34545455, 0.49230769, 0.78461538, 1.06666667, 1.2       ,
       1.        , 1.5       , 0.89361702, 1.22857143, 0.69444444,
       0.75384615, 0.7       , 0.6875    , 0.96969697, 0.64285714,
       1.        , 0.52459016, 0.76785714, 0.71052632, 1.14782609,
       0.36842105, 0.71171171, 0.68      , 0.42857143, 0.54166667,
       0.55629139, 0.4       , 0.36666667, 0.70945946, 0.5       ,
...
       0.95652174, 0.66666667, 0.71428571, 1.16666667, 1.42372881,
       0.85365854, 1.1025641 , 0.90909091, 2.625     , 0.94160584,
       1.22222222, 1.        , 0.94871795, 0.43137255, 1.60135135,
       1.08      , 1.        , 1.17808219, 0.75641026, 0.67857143,
       0.8125    , 1.13253012, 0.90697674, 0.88461538, 1.20833333,
       0.97938144, 1.10576923, 1.3559322 , 1.0619469 , 0.5       ,
       1.2375    , 0.98809524, 0.83157895, 0.8       , 0.98333333,
       0.73015873, 0.5890411 , 0.79411765, 0.54347826, 0.4       ,
       0.60747664, 0.38095238, 0.21978022, 0.56875   , 0.59459459,
       0.3164557 , 0.79166667, 1.05357143, 0.        , 1.05228758,
       0.5       , 1.23417722, 0.71830986, 0.56481481, 0.97716895,
       0.79245283, 0.82758621, 0.65116279, 0.83333333, 0.86666667,
       0.71779141, 1.15384615, 1.01388889, 0.72580645, 0.86842105,
       1.        , 0.7       , 1.84210526, 1.22222222, 1.37931034,
       0.91666667, 0.98876404, 0.69747899, 1.0052356 , 0.81578947,
       1.125     , 0.64705882, 0.76056338, 0.74576271, 0.87777778,
       0.92142857, 1.03333333, 0.65384615, 0.92307692, 1.        ,
       1.1625    , 0.62637363, 1.10714286, 1.07352941, 1.09375   ,
       1.14285714, 0.86440678, 1.30645161, 0.8071066 , 0.87878788,
       1.        , 0.80898876, 0.89795918, 0.58974359])
Coordinates:
  * sample   (sample) object '1000_1001' '1001_1002' ... '99_99' '9_9'
    gene     <U6 'VPREB3'

In [93]:
[y.gene.values][0]

array('VPREB3', dtype='<U6')

In [91]:
y_df = pd.DataFrame(data = y.values.reshape(y.shape[0], 1),
                   index = y.sample.values, columns = [gene_name])
y_df.head()

,VPREB3
1000_1001,0.971154
1001_1002,0.970588
1002_1003,1.148936
1003_1004,1.258065
1004_1005,1.093750


In [48]:
y = pd.read_csv(y_file, sep="\t")
y

,sampleid,RP11-34P13.7,RP11-34P13.8,RP11-34P13.13,RP11-206L10.4,LINC01409,FAM87B,LINC01128,LINC00115,RP11-206L10.19,...,MT-CO2,MT-ATP8,MT-ATP6,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-CYB
0,686_687,0.0,0.0,0.000000,0.0,0.000000,0.0,0.008850,0.008850,0.000000,...,11.991150,0.026549,6.973451,12.230088,9.398230,0.610619,9.061947,1.938053,0.115044,8.380531
1,682_683,0.0,0.0,0.000000,0.0,0.061224,0.0,0.000000,0.000000,0.020408,...,16.265306,0.061224,9.346939,15.020408,9.387755,0.673469,10.142857,1.795918,0.102041,7.408163
2,692_693,0.0,0.0,0.020408,0.0,0.020408,0.0,0.020408,0.000000,0.020408,...,12.653061,0.020408,7.204082,10.612245,9.224490,0.571429,10.326531,1.836735,0.102041,6.081633
3,683_684,0.0,0.0,0.000000,0.0,0.007519,0.0,0.015038,0.022556,0.007519,...,14.887218,0.067669,8.827068,13.255639,10.646617,0.669173,11.270677,2.684211,0.082707,9.375940
4,687_688,0.0,0.0,0.000000,0.0,0.029412,0.0,0.000000,0.000000,0.000000,...,16.073529,0.073529,10.073529,13.897059,9.573529,1.000000,11.794118,2.544118,0.029412,9.470588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,799_800,0.0,0.0,0.000000,0.0,0.064516,0.0,0.000000,0.000000,0.032258,...,15.935484,0.064516,9.290323,18.548387,9.774194,0.548387,13.322581,2.193548,0.096774,9.129032
965,822_823,0.0,0.0,0.000000,0.0,0.000000,0.0,0.029412,0.000000,0.117647,...,17.382353,0.000000,8.088235,14.764706,9.147059,0.911765,12.676471,2.264706,0.088235,8.029412
966,840_841,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.050000,...,14.700000,0.000000,9.050000,18.050000,9.550000,0.700000,10.850000,2.250000,0.150000,10.500000
967,814_815,0.0,0.0,0.000000,0.0,0.028571,0.0,0.057143,0.000000,0.028571,...,19.828571,0.142857,11.885714,19.457143,13.057143,0.657143,13.314286,2.228571,0.028571,10.657143


In [49]:
import re
for column in y.columns:
    if re.search("IGLL", column):
        print(column)

IGLL1


In [50]:
len(y.columns)

27818

In [66]:
y.loc[:,["sampleid","VPREB3"]].values

array([['686_687', 0.92920353982301],
       ['682_683', 0.693877551020408],
       ['692_693', 1.02040816326531],
       ...,
       ['840_841', 0.95],
       ['814_815', 0.114285714285714],
       ['821_822', 1.31578947368421]], dtype=object)

In [74]:
y_df = pd.DataFrame(data = y.loc[:,["sampleid","VPREB3"]].values[:,1], index=y.loc[:,["sampleid","VPREB3"]].values[:,0], columns = ["VPREB3"])
y_df.head()

,VPREB3
686_687,0.929204
682_683,0.693878
692_693,1.020408
683_684,0.796992
687_688,0.985294


In [75]:
y_df.to_csv("gs://cpg-tob-wgs-test/v0/vpreb3_B_naive_expression.csv")

2022-09-27 01:51:21 DEBUG (gcsfs 384): POST: https://storage.googleapis.com/upload/storage/v1/b/cpg-tob-wgs-test/o, (), {'X-Upload-Content-Type': 'application/octet-stream'}
2022-09-27 01:51:22 DEBUG (gcsfs 384): POST: https://storage.googleapis.com/upload/storage/v1/b/cpg-tob-wgs-test/o?uploadType=resumable&upload_id=ADPycdtRMqghP9yX8Xwy6VQB8a0SefCSonClCu5mowNVcunNFR89W1SHuftLr7NqJCIadetO2FvI7volP-drD_4_kEr6qA, (), {'Content-Range': 'bytes 0-22592/22593', 'Content-Type': 'application/octet-stream', 'Content-Length': '22593'}


In [52]:
# y[["sampleid"]]

In [53]:
# genotype_file_bed = "gs://cpg-tob-wgs-test/v0/plink_files/vpreb3_rare_regulatory.bed"
# genotype_file_bim = "gs://cpg-tob-wgs-test/v0/plink_files/vpreb3_rare_regulatory.bim"
# genotype_file_fam = "gs://cpg-tob-wgs-test/v0/plink_files/vpreb3_rare_regulatory.fam"

In [4]:
genotype_file_bed = "gs://cpg-tob-wgs-test/v0/plink_files/vpreb3_rare_promoter.bed"
genotype_file_bim = "gs://cpg-tob-wgs-test/v0/plink_files/vpreb3_rare_promoter.bim"
genotype_file_fam = "gs://cpg-tob-wgs-test/v0/plink_files/vpreb3_rare_promoter.fam"

# bed
with to_path(genotype_file_bed).open('rb') as handle:
    data = handle.readlines()
with open('temp.bed', 'wb') as handle:
    handle.writelines(data)
# bim
with to_path(genotype_file_bim).open('rb') as handle:
    data = handle.readlines()
with open('temp.bim', 'wb') as handle:
    handle.writelines(data)
# fam
with to_path(genotype_file_fam).open('rb') as handle:
    data = handle.readlines()
with open('temp.fam', 'wb') as handle:
    handle.writelines(data)
# read
G = read_plink1_bin('temp.bed')

In [94]:
G

<xarray.DataArray 'genotype' (sample: 1061, variant: 185)>
dask.array<transpose, shape=(1061, 185), dtype=float32, chunksize=(1024, 185), chunktype=numpy.ndarray>
Coordinates:
  * sample   (sample) object 'CPG18' 'CPG26' 'CPG34' ... 'NA12891' 'NA12892'
  * variant  (variant) <U10 'variant0' 'variant1' ... 'variant183' 'variant184'
    fid      (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    iid      (sample) object 'CPG18' 'CPG26' 'CPG34' ... 'NA12891' 'NA12892'
    father   (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    mother   (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    gender   (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    trait    (sample) object <NA> <NA> <NA> <NA> <NA> ... <NA> <NA> <NA> <NA>
    chrom    (variant) object 'chr22' 'chr22' 'chr22' ... 'chr22' 'chr22'
    snp      (variant) object 'chr22:23715912:G:A' ... 'chr22:23788386:T:C'
    cm       (variant) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    pos      (variant) int32 23715912 23715916 23715930 ... 23788289 23788386
    a0       (variant) object 'A' 'A' 'G' 'T' 'G' 'T' ... 'G' 'T' 'C' 'G' 'C'
    a1       (variant) object 'G' 'C' 'A' 'G' 'A' 'C' ... 'A' 'C' 'T' 'A' 'T'

In [174]:
# G = G.sel(sample=donors1)
# G.shape

In [7]:
import numpy as np
np.nanmin(G.values)

0.0

In [8]:
np.nanmean(G.values)

1.9940945

In [9]:
np.nanmax(G.values)

2.0

In [10]:
data = G.values
Z_df = pd.DataFrame(data, columns=G.snp.values, index=G.sample.values)
Z_df.head()

,chr22:23715912:G:A,chr22:23715916:C:A,chr22:23715930:A:G,chr22:23715973:G:T,chr22:23716018:A:G,chr22:23716098:C:T,chr22:23716279:C:G,chr22:23716292:C:T,chr22:23716297:C:T,chr22:23716298:G:A,...,chr22:23787582:C:T,chr22:23787591:T:C,chr22:23787608:T:C,chr22:23787642:C:T,chr22:23788084:G:A,chr22:23788178:A:G,chr22:23788250:C:T,chr22:23788278:T:C,chr22:23788289:A:G,chr22:23788386:T:C
CPG18,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
CPG26,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
CPG34,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
CPG42,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
CPG59,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [11]:
Z_df = Z_df.dropna(axis=1)
Z_df.shape

(1061, 177)

In [12]:
Z_df.min(axis=1)

CPG18       2.0
CPG26       1.0
CPG34       1.0
CPG42       1.0
CPG59       2.0
           ... 
CPG67744    2.0
syndip      1.0
NA12878     1.0
NA12891     1.0
NA12892     2.0
Length: 1061, dtype: float32

In [13]:
# Z_df.to_csv("gs://cpg-tob-wgs-test/v0/vpreb3_rare_regulatory.csv")
Z_df.to_csv("gs://cpg-tob-wgs-test/v0/vpreb3_rare_promoter.csv")

2022-09-27 04:30:54 DEBUG (asyncio 58): Using selector: EpollSelector
2022-09-27 04:30:54 DEBUG (google.auth._default 204): Checking None for explicit credentials as part of auth process...
2022-09-27 04:30:54 DEBUG (google.auth._default 181): Checking Cloud SDK credentials as part of auth process...
2022-09-27 04:30:54 DEBUG (google.auth._default 187): Cloud SDK credentials not found on disk; not using them
2022-09-27 04:30:54 DEBUG (google.auth._default 223): Checking for App Engine runtime as part of auth process...
2022-09-27 04:30:54 DEBUG (google.auth._default 235): No App Engine library was found so cannot authentication via App Engine Identity Credentials.
2022-09-27 04:30:54 DEBUG (google.auth.transport._http_client 104): Making request: GET http://169.254.169.254
2022-09-27 04:30:54 DEBUG (google.auth.transport._http_client 104): Making request: GET http://metadata.google.internal/computeMetadata/v1/project/project-id
2022-09-27 04:30:54 DEBUG (gcsfs.credentials 217): Connect